In [52]:
import xml.etree.ElementTree as ET
import pyodbc
from psycopg2 import Error

tree = ET.parse('config.xml')
root = tree.getroot()
db_cred=''
pg_prefix='AM_'
source_t = '''SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH 
                FROM %(db_name)s.INFORMATION_SCHEMA.COLUMNS where TABLE_SCHEMA='%(s_name)s' and TABLE_NAME='%(t_name)s' '''
q_create_target_db= '''create database %(db_name)s'''
for sourse in root:
    print(sourse.tag, sourse.attrib)

    for db_name in sourse:

        if db_name.tag=='cred' :
            db_cred=db_name.text
        if db_name.tag=='target' :
            db_cred_target_first=db_name.text + 'Database=' + db_name.attrib['db_default']
            print('\nTarget DB: ',db_cred_target_first)
            try:
                t_conn = pyodbc.connect(db_cred_target_first)
                print("\nConnection success to target DB: ", db_name.attrib['db_default'],'\n')
            except (Exception, Error) as error:
                print("\nError connection! ", error)


        if db_name.tag=='DB' :
            print(db_name.tag, db_name.attrib)
            db_cred = db_cred + 'Database=' + db_name.attrib['name']
            print('\nSourse DB: ',db_cred)

            try:
                d_conn = pyodbc.connect(db_cred)
                print("Connection success to sourse DB: ", db_name.attrib['name'])
            except (Exception, Error) as error:
                print("\nError connection! ", error)   

           

            for s_name in db_name :
                print(s_name.tag, s_name.attrib)

                for t_name in s_name :
                    print(t_name.tag, t_name.attrib)
                    print('\nGetting inform sourse.')
                    t_name_param = {
                        'db_name' : db_name.attrib['name'] ,
                        's_name' : s_name.attrib['name'] ,
                        't_name' : t_name.attrib['name'] 
                    }
                    source_t_q = source_t % t_name_param
                    print(source_t_q)
                    cursor_t = d_conn.cursor()
                    cursor_t.execute(source_t_q)
                    for i in cursor_t:
                        print(i)
                    

                    



            db_cred=''



sourse {'name': 'MS_SQL', 'type': 'SQL Server'}

Target DB:  DRIVER={PostgreSQL Unicode};Server=localhost;port=5432;UID=for_py1;PWD=for_py1;Database=postgres

Connection success to target DB:  postgres 

DB {'name': 'DB_DEMO_SDBO'}

Sourse DB:  Driver={SQL Server};Server=DESKTOP-NOR5JCO\SQLSERVER2019;UID=for_py1;PWD=for_py1;Database=DB_DEMO_SDBO
Connection success to sourse DB:  DB_DEMO_SDBO
schema {'name': 'dbo'}
table {'name': 'CSV_SER_SPR'}

Getting inform sourse.
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH 
                FROM DB_DEMO_SDBO.INFORMATION_SCHEMA.COLUMNS where TABLE_SCHEMA='dbo' and TABLE_NAME='CSV_SER_SPR' 
('SER_ID', 'nvarchar', 50)
('SER_NAME', 'nvarchar', 100)
('SER_DATE_START', 'date', None)
table {'name': 'CSV_SYSTEM'}

Getting inform sourse.
SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH 
                FROM DB_DEMO_SDBO.INFORMATION_SCHEMA.COLUMNS where TABLE_SCHEMA='dbo' and TABLE_NAME='CSV_SYSTEM' 
('SYS_ID', 'nvarchar', 50)
('SYS_NAME', 